In [3]:
import sys
sys.path.append('/home/zuzanna/Desktop/gan_aug/gan_compare')

from gan_compare.paths import INBREAST_IMAGE_PATH, INBREAST_XML_PATH, INBREAST_CSV_PATH
from gan_compare.data_utils.utils import load_inbreast_mask, get_file_list, read_csv

from typing import Tuple
from pathlib import Path
import os.path
import glob
import cv2
import numpy as np
import pandas as pd
import json
import argparse
from tqdm import tqdm
import pydicom as dicom

In [5]:
metadata_path = "/home/zuzanna/Desktop/gan_aug/gan_compare/metadata/metadata.json"

In [6]:
with open(metadata_path, "r") as metadata_file:
    metadata = json.load(metadata_file)

In [7]:
len(metadata)

2628

In [8]:
import pandas as pd

In [11]:
df = pd.read_json(metadata_path)
df

,image_id,patient_id,ACR,birads,laterality,view,lesion_id,bbox,image_path,xml_path
0,51048972,f3e93e889a7746f0,3,2,R,CC,4,"[3306, 1332, 19, 21]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...
1,50993643,b03f1dd34eb3c55f,2,2,L,CC,1,"[1061, 1970, 23, 23]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...
2,51049628,6f64793857feb5d0,2,2,L,MLO,5,"[746, 1683, 9, 14]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...
3,20587080,b6a4f750c6df4f90,2,4c,R,MLO,0,"[1963, 2026, 68, 66]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...
4,20587080,b6a4f750c6df4f90,2,4c,R,MLO,1,"[2046, 1952, 134, 129]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...
...,...,...,...,...,...,...,...,...,...,...
2623,50997107,ce5e5e18a261cd29,1,2,R,MLO,99,"[1572, 1266, 4, 6]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...
2624,50997107,ce5e5e18a261cd29,1,2,R,MLO,100,"[1581, 1259, 6, 5]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...
2625,50997107,ce5e5e18a261cd29,1,2,R,MLO,101,"[1510, 1252, 11, 11]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...
2626,50997107,ce5e5e18a261cd29,1,2,R,MLO,102,"[1488, 1252, 5, 3]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...


In [18]:
df["birads_int"] = df.apply(lambda row: int(row["birads"][0]), axis=1)

In [19]:
df

,image_id,patient_id,ACR,birads,laterality,view,lesion_id,bbox,image_path,xml_path,birads_int
0,51048972,f3e93e889a7746f0,3,2,R,CC,4,"[3306, 1332, 19, 21]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,2
1,50993643,b03f1dd34eb3c55f,2,2,L,CC,1,"[1061, 1970, 23, 23]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,2
2,51049628,6f64793857feb5d0,2,2,L,MLO,5,"[746, 1683, 9, 14]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,2
3,20587080,b6a4f750c6df4f90,2,4c,R,MLO,0,"[1963, 2026, 68, 66]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,4
4,20587080,b6a4f750c6df4f90,2,4c,R,MLO,1,"[2046, 1952, 134, 129]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,4
...,...,...,...,...,...,...,...,...,...,...,...
2623,50997107,ce5e5e18a261cd29,1,2,R,MLO,99,"[1572, 1266, 4, 6]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,2
2624,50997107,ce5e5e18a261cd29,1,2,R,MLO,100,"[1581, 1259, 6, 5]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,2
2625,50997107,ce5e5e18a261cd29,1,2,R,MLO,101,"[1510, 1252, 11, 11]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,2
2626,50997107,ce5e5e18a261cd29,1,2,R,MLO,102,"[1488, 1252, 5, 3]",/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,/home/zuzanna/Desktop/gan_aug/InBreast/InBreas...,2


In [20]:
df["birads_int"].describe()

count    2628.000000
mean        2.486682
std         1.011268
min         2.000000
25%         2.000000
50%         2.000000
75%         2.000000
max         6.000000
Name: birads_int, dtype: float64

In [21]:
df.describe()

,image_id,ACR,lesion_id,birads_int
count,2.628000e+03,2628.000000,2628.000000,2628.000000
mean,4.236038e+07,2.056317,38.176941,2.486682
std,1.337404e+07,0.858834,38.057880,1.011268
min,2.058691e+07,1.000000,0.000000,2.000000
25%,2.405538e+07,1.000000,7.000000,2.000000
50%,5.099390e+07,2.000000,24.000000,2.000000
75%,5.099713e+07,3.000000,63.000000,2.000000
max,5.358772e+07,4.000000,169.000000,6.000000


In [23]:
len(df[df["birads_int"]==2])

2059

In [24]:
len(df[df["birads_int"]==3])

153

In [25]:
len(df[df["birads_int"]==4])

133

In [26]:
len(df[df["birads_int"]==5])

272

In [27]:
len(df[df["birads_int"]==6])

11